# 图像卷积

---

In [4]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    '''计算二维互相关运算'''
    h, w = K.shape #K是核矩阵
    Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1) #Y是输出
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h,j:j + w] * K).sum()
    return Y

## 验证上述二维互相关运算的输出

In [5]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 实现二维卷积层

In [7]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 卷积层的一个简单应用：
检测图像中不同颜色的边缘

In [8]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
K = torch.tensor([[1.0, -1.0]])  #1*2的一个和 罗伯特算子

 输出Y中的1代表从白色到黑色的边缘 ，-1代表从黑色到白色的边缘

In [10]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核`K`只可以检测`垂直的`边缘

In [11]:
corr2d(X.t(), K)#将X转置后就不能检测了

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

上面是知道X（输入） K（卷积核）得到Y<br>
下面是知道X（输入）Y（输出），去学习是一个什么样子的K（卷积核）

In [13]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
'''Conv2d是一个四维的东西，所以要改变一下XY
第一个是通道维：通道数，RGB=3.灰度图=1，
第二个是批量大小维：就是样本数'''
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad #这里是梯度下降
    if (i + 1) % 2 == 0:
        print(f'batch {i + 1}, loss {l.sum():.3f}')

batch 2, loss 8.394
batch 4, loss 1.475
batch 6, loss 0.275
batch 8, loss 0.057
batch 10, loss 0.014


In [17]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9761, -0.9943]])